In [36]:
import sqlite3
import yaml
import json
from monad_std import Option
from typing import *
from dataclasses import dataclass
import cbor2

In [2]:
db = sqlite3.connect("./skills.db")
cursor = db.cursor()

In [5]:
cursor.execute("""create table if not exists recursive (
    type_id integer,
    total integer,
    prereq blob
);""")
db.commit()

In [55]:
@dataclass(kw_only=True)
class SkillObject:
    skill_id: int
    skill_level: int
    factor: int
    
    @staticmethod
    def get_skill(type_id: int) -> Option[List["SkillObject"]]:
        cursor.execute("select skill_id, skill_level from skills where type_id == ?", (type_id,))
        res = cursor.fetchall()
        if not res:
            return Option.none()
        else:
            def process(val: List[int]) -> SkillObject:
                cursor.execute("select time_factor from skill_constant where skill_id == ?;", (val[0],))
                r = cursor.fetchone()
                if r:
                    result =  SkillObject(skill_id=val[0], skill_level=val[1], factor=r[0])
                else:
                    result = SkillObject(skill_id=val[0], skill_level=val[1], factor=0)
                return result
            return Option.some(list(map(process, res)))

@dataclass(kw_only=True)
class SkillGroup:
    skill: SkillObject
    prerequisites: List["SkillGroup"]
    
    @staticmethod
    def recursive_get(type_id: int, type_id_set: Set[int]) -> Option[List["SkillGroup"]]:
        skills = SkillObject.get_skill(type_id)
        if skills.is_none():
            return Option.none()
        else:
            ls = []
            for skill in skills.unwrap():
                if skill.skill_id in type_id_set:
                    continue
                cloned = set(type_id_set)
                cloned.add(skill.skill_id)
                sk = SkillGroup.recursive_get(skill.skill_id, cloned)
                sg = SkillGroup(skill=skill, prerequisites=sk.unwrap_or([]))
                ls.append(sg)
            return Option.some(ls)

@dataclass(kw_only=True)
class Skill:
    type_id: int
    group: List[SkillGroup]
    
    @staticmethod
    def get_skill_recursive(type_id: int) -> Option["Skill"]:
        s = set()
        return SkillGroup.recursive_get(type_id, s).map(lambda x: Skill(type_id=type_id, group=x))
    
    @property
    def total_points(self) -> int:
        skill_time_factor = [0, 250, 1414, 8000, 45254, 256000]
        level_dict: Dict[int, int] = {}
        factor_dict: Dict[int, int] = {}
        
        def __rec(grp: SkillGroup):
            nonlocal level_dict
            if grp.skill.skill_id not in factor_dict.keys():
                factor_dict[grp.skill.skill_id] = grp.skill.factor
            
            if grp.skill.skill_id not in level_dict.keys():
                level_dict[grp.skill.skill_id] = grp.skill.skill_level
            elif level_dict[grp.skill.skill_id] < grp.skill.skill_level:
                level_dict[grp.skill.skill_id] = grp.skill.skill_level
            for p in grp.prerequisites:
                __rec(p)
                
        for g in self.group:
            __rec(g)
        
        points = sum(map(lambda item: skill_time_factor[item[1]] * factor_dict[item[0]], level_dict.items()), start=0)
        return points

In [29]:
def create_as_dict(val: Skill) -> dict:
    def ser(grp: SkillGroup) -> dict:
        return {
            "id": grp.skill.skill_id,
            "level": grp.skill.skill_level,
            "prereq": list(map(ser, grp.prerequisites))
        }
    return list(map(ser, val.group))

In [57]:
cursor.execute("""select type_id from skills;""")
__sql = "insert into recursive values (?, ?, ?);"
type_id_list = cursor.fetchall()
for type_id, in type_id_list:
    skill = Skill.get_skill_recursive(type_id).unwrap()
    d = create_as_dict(skill)
    tid = skill.type_id
    point = skill.total_points
    cursor.execute(__sql, (tid, point, cbor2.dumps(d)))
db.commit()